In [1]:
import pandas as pd
import numpy as np
import json

def extract_address_components(address):
    # Extracting components from the nested address dictionary
    street = address.get("street", None)
    suburb = address.get("suburb", None)
    government_area = address.get("government_area", None)
    market = address.get("market", None)
    country = address.get("country", None)
    country_code = address.get("country_code", None)

    # Extracting nested location information
    location = address.get("location", {})
    location_type = location.get("type", None)
    coordinates = location.get("coordinates", [None, None])
    latitude = coordinates[1]
    longitude = coordinates[0]
    is_location_exact = location.get("is_location_exact", None)

    return (
        street,
        suburb,
        government_area,
        market,
        country,
        country_code,
        latitude,
        longitude,
        is_location_exact,
        location_type,
    )


def sortAmenties(amenites):
    a = amenites
    a.sort(reverse=False)
    return a


def extractHostinfo(hostDetails):
    host_id = hostDetails.get("host_id", None)
    host_url = hostDetails.get("host_url", None)
    host_name = hostDetails.get("host_name", None)
    host_identity_verified = hostDetails.get("host_identity_verified", None)
    host_location = hostDetails.get("host_location", None)
    host_response_time = hostDetails.get("host_response_time", None)
    host_thumbnail_url = hostDetails.get("host_thumbnail_url", None)
    host_picture_url = hostDetails.get("host_picture_url", None)
    host_neighbourhood = hostDetails.get("host_neighbourhood", None)
    host_response_rate = hostDetails.get("host_response_rate", None)
    host_is_superhost = hostDetails.get("host_is_superhost", None)
    host_has_profile_pic = hostDetails.get("host_has_profile_pic", None)
    host_listings_count = hostDetails.get("host_listings_count", None)
    host_total_listings_count = hostDetails.get("host_total_listings_count", None)
    host_verifications = hostDetails.get("host_verifications", None)
    return (
        host_id,
        host_url,
        host_name,
        host_identity_verified,
        host_location,
        host_response_time,
        host_thumbnail_url,
        host_picture_url,
        host_neighbourhood,
        host_response_rate,
        host_is_superhost,
        host_has_profile_pic,
        host_listings_count,
        host_total_listings_count,
        host_verifications,
    )


def extract_availability(availability):
    availability_365 = availability.get("availability_365", None)
    availability_30 = availability.get("availability_30", None)
    availability_60 = availability.get("availability_60", None)
    availability_90 = availability.get("availability_90", None)
    return availability_365, availability_30, availability_60, availability_90


def percentage_to_stars(percentage):
    if percentage is None:
        return None
    return (percentage / 100) * 5


def extract_rating(review_scores):
    rating = percentage_to_stars(review_scores.get("review_scores_rating", None))
    return rating


def processValues(value):
    if value is None:
        return "Not specified"
    else:
        return value
    
def processnanValues(value):
    if value == "nan":
        return "Not specified"
    else:
        return value
  

In [2]:
airbnb = pd.read_json("sample_airbnb.json")

In [3]:


# date format conversion
airbnb["last_scraped"] = pd.to_datetime(
    airbnb["last_scraped"], format="%Y-%m-%d %H:%M:%S"
)
airbnb["calendar_last_scraped"] = pd.to_datetime(
    airbnb["calendar_last_scraped"], format="%Y-%m-%d %H:%M:%S"
)
airbnb["first_review"] = pd.to_datetime(
    airbnb["first_review"], format="%Y-%m-%d %H:%M:%S"
)
airbnb["last_review"] = pd.to_datetime(
    airbnb["last_review"], format="%Y-%m-%d %H:%M:%S"
)

airbnb["first_review"] = airbnb["first_review"].dt.strftime("%Y-%m-%d")
airbnb["last_review"] = airbnb["last_review"].dt.strftime("%Y-%m-%d")
airbnb["last_scraped"] = airbnb["last_scraped"].dt.strftime("%Y-%m-%d")
airbnb["calendar_last_scraped"] = airbnb["calendar_last_scraped"].dt.strftime(
    "%Y-%m-%d"
)

airbnb["first_review"] = pd.to_datetime(airbnb["first_review"], format="%Y-%m-%d")
airbnb["last_review"] = pd.to_datetime(airbnb["last_review"], format="%Y-%m-%d")
airbnb["last_scraped"] = pd.to_datetime(airbnb["last_scraped"], format="%Y-%m-%d")
airbnb["calendar_last_scraped"] = pd.to_datetime(
    airbnb["calendar_last_scraped"], format="%Y-%m-%d"
)

# Apply the custom function to the 'address' column and expand the result into multiple columns
airbnb[
    [
        "street",
        "suburb",
        "government_area",
        "market",
        "country",
        "country_code",
        "latitude",
        "longitude",
        "is_location_exact",
        "location_type",
    ]
] = airbnb["address"].apply(lambda x: pd.Series(extract_address_components(x)))

# extracting host information
airbnb[
    [
        "host_id",
        "host_url",
        "host_name",
        "host_identity_verified",
        "host_location",
        "host_response_time",
        "host_thumbnail_url",
        "host_picture_url",
        "host_neighbourhood",
        "host_response_rate",
        "host_is_superhost",
        "host_has_profile_pic",
        "host_listings_count",
        "host_total_listings_count",
        "host_verifications",
    ]
] = airbnb["host"].apply(lambda x: pd.Series(extractHostinfo(x)))

# Extracting Availabilities
airbnb[
    ["availability_365", "availability_30", "availability_60", "availability_90"]
] = airbnb["availability"].apply(lambda x: pd.Series(extract_availability(x)))

# Rating
airbnb[["rating"]] = airbnb["review_scores"].apply(
    lambda x: pd.Series(extract_rating(x))
)
# amenties
airbnb["amenities"] = airbnb["amenities"].apply(lambda x: sortAmenties(x))

# host_verification
airbnb["host_verifications"] = airbnb["host_verifications"].apply(
    lambda x: sortAmenties(x)
)

airbnb["amenities"] = airbnb["amenities"].apply(json.dumps)
airbnb["host_verifications"] = airbnb["host_verifications"].apply(json.dumps)

# Street handling
airbnb["street"] = airbnb["street"].str.split(",").str[0]

#  Impute missing data
airbnb = airbnb.fillna({"reviews_per_month": 0, "rating": 0})

# Create is_rated column
is_rated = np.where(airbnb["rating"].isna() == True, 0, 1)
airbnb["is_rated"] = is_rated

# Image url
airbnb["images"] = airbnb["images"].apply(lambda x: x["picture_url"])

# filling nan values
airbnb["bedrooms"].fillna(0, inplace=True)
airbnb["beds"].fillna(0, inplace=True)
airbnb["bathrooms"].fillna(0, inplace=True)
airbnb['bathrooms'].round()

# converting data types
airbnb["latitude"] = airbnb["latitude"].apply(lambda x: float(x))
airbnb["longitude"] = airbnb["longitude"].apply(lambda x: float(x))
airbnb["price"] = airbnb["price"].astype("float")
airbnb["weekly_price"] = airbnb["weekly_price"].astype("float")
airbnb["cleaning_fee"] = airbnb["cleaning_fee"].astype("float")
airbnb["minimum_nights"] = airbnb["minimum_nights"].astype(int)
airbnb["maximum_nights"] = airbnb["maximum_nights"].astype(int)
airbnb["bedrooms"] = airbnb["bedrooms"].astype(int)
airbnb["beds"] = airbnb["beds"].astype(int)
airbnb["bathrooms"] = airbnb["bathrooms"].astype(str).astype(float)
airbnb["price"] = airbnb["price"].astype(str).astype(float).astype(int)
airbnb["extra_people"] = (
    airbnb["extra_people"].astype(str).astype(float).astype(int)
)
airbnb["guests_included"] = airbnb["guests_included"].astype(str).astype(int)

# Processing Nan values to Not specified
airbnb["host_response_time"] = airbnb["host_response_time"].apply(
    lambda x: processValues(x)
)
airbnb['host_response_rate'] = airbnb['host_response_rate'].astype(str)
airbnb["host_response_rate"] = airbnb["host_response_rate"].apply(
    lambda x: processnanValues(x)
)

# Processing true & false to Yes & no
airbnb["host_is_superhost"] = airbnb["host_is_superhost"].map(
    {False: "No", True: "Yes"}
)
airbnb["host_has_profile_pic"] = airbnb["host_has_profile_pic"].map(
    {False: "No", True: "Yes"}
)
airbnb["host_identity_verified"] = airbnb["host_identity_verified"].map(
    {False: "No", True: "Yes"}
)
airbnb["is_location_exact"] = airbnb["is_location_exact"].map(
    {False: "No", True: "Yes"}
)

# Filling Security deposit with median values
sd_median_value = airbnb["security_deposit"].median()
cf_median_value = airbnb["cleaning_fee"].median()
airbnb["security_deposit"] = airbnb["security_deposit"].fillna(sd_median_value)
airbnb["cleaning_fee"] = airbnb["cleaning_fee"].fillna(cf_median_value)

# Using forward fill to fill first and last review
airbnb["first_review"] = airbnb["first_review"].ffill()
airbnb["last_review"] = airbnb["last_review"].ffill()
airbnb = airbnb.map(
    lambda x: "Not Specified" if isinstance(x, str) and x.strip() == "" else x
)

airbnb.drop("host", axis=1, inplace=True)
airbnb.drop("availability", axis=1, inplace=True)
airbnb.drop("reviews", axis=1, inplace=True)
airbnb.drop("address", axis=1, inplace=True)
airbnb.drop("weekly_price", axis=1, inplace=True)
airbnb.drop("monthly_price", axis=1, inplace=True)
airbnb.drop("review_scores", axis=1, inplace=True)

C:\Users\faiza\AppData\Local\Temp\ipykernel_28332\1414383994.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airbnb["bedrooms"].fillna(0, inplace=True)
C:\Users\faiza\AppData\Local\Temp\ipykernel_28332\1414383994.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [7]:
airbnb.rename(columns={'_id': 'id'}, inplace=True)

In [5]:
airbnb.to_csv("airbnbnew1.csv", index=False)

In [8]:
import pandas as pd

In [3]:
airbnb = pd.read_csv("airbnbnew1.csv")

In [5]:
airbnb

,_id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,host_has_profile_pic,host_listings_count,host_total_listings_count,host_verifications,availability_365,availability_30,availability_60,availability_90,rating,is_rated
0,1003530,https://www.airbnb.com/rooms/1003530,New York City - Upper West Side Apt,Not Specified,"Murphy bed, optional second bedroom available....","Murphy bed, optional second bedroom available....",Great neighborhood - many terrific restaurants...,"My cat, Samantha, are in and out during the su...","Conveniently located near 1, 2, 3, B & C subwa...",New York City!,...,Yes,3,3,"[""email"", ""government_id"", ""jumio"", ""offline_g...",93,0,0,0,4.70,1
1,10133554,https://www.airbnb.com/rooms/10133554,Double and triple rooms Blue mosque,Not Specified,We are on the central city Blue mosque 5 minu...,We are on the central city Blue mosque 5 minu...,Cankurtaran mahallesi akbıyık caddesi no 22 Su...,Not Specified,Tram 5 minutes metro 10 minutes Bus 10 minutes,"We have bathroom,wc,İnternet in rooms and unde...",...,Yes,2,2,"[""email"", ""facebook"", ""government_id"", ""jumio""...",365,30,60,90,4.60,1
2,10059872,https://www.airbnb.com/rooms/10059872,"Soho Cozy, Spacious and Convenient","Clean, fully furnish, Spacious 1 bedroom flat ...",Not Specified,"Clean, fully furnish, Spacious 1 bedroom flat ...",Not Specified,Not Specified,Not Specified,Not Specified,...,Yes,1,1,"[""email"", ""government_id"", ""jumio"", ""phone"", ""...",0,0,0,0,5.00,1
3,10084023,https://www.airbnb.com/rooms/10084023,City center private room with bed,House is located 5mins walk from Sham Shui Po ...,"The house is old fashion type, and paint the w...",House is located 5mins walk from Sham Shui Po ...,"Cheapest food, electronic device, clothing and...",Deposit of $1000 will be charged and will retu...,"Close to 3 different MTR Station, Sham shui Po...","Living Room , Kitchen and Toilet, All cooking ...",...,Yes,2,2,"[""email"", ""government_id"", ""jumio"", ""offline_g...",220,14,24,40,4.60,1
4,10091713,https://www.airbnb.com/rooms/10091713,Surry Hills Studio - Your Perfect Base in Sydney,"This spacious, light filled studio has everyth...",Comfortable studio with a great layout. The ba...,"This spacious, light filled studio has everyth...",Surry Hills is a vibrant eclectic community an...,"WiFi, Apple TV with Netflix App (for use with ...",Travel Card 'Opal' travel cards work on all Sy...,You have full use of the entire studio and com...,...,Yes,1,1,"[""email"", ""google"", ""government_id"", ""jumio"", ...",0,0,0,0,4.75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5550,9985696,https://www.airbnb.com/rooms/9985696,Kadıköy-Altıyol,Kadıköy merkezde 2+1 dairem vardır.Çok kalabal...,Not Specified,Kadıköy merkezde 2+1 dairem vardır.Çok kalabal...,Not Specified,Not Specified,Not Specified,Not Specified,...,Yes,2,2,"[""email"", ""phone""]",365,30,60,90,0.00,1
5551,9993190,https://www.airbnb.com/rooms/9993190,Just Steps from the Ocean at Premier Platinum ...,Relax Hawaiian Style in Gated Community on Oce...,"***NOTE: THE OCEAN INFINITY POOL (""POOL 1""0 WI...",Relax Hawaiian Style in Gated Community on Oce...,Location: Located at the south end of Lahaina ...,Puamana Resort requires a $10/day resort fee w...,You can walk to most attractions within Lahain...,Guests will have access to the 3 pools in the ...,...,Yes,58,58,"[""email"", ""kba"", ""phone"", ""reviews"", ""work_ema...",258,0,8,31,4.60,1
5552,9887054,https://www.airbnb.com/rooms/9887054,Quarry Bay fully furnished studio C,Newly renovated studio flat with full furniture.,Not Specified,Newly renovated studio flat with full furniture.,Not Specified,Not Specified,Not Specified,Not Specified,...,Yes,7,7,"[""email"", ""phone"", ""reviews""]",353,18,48,78,4.25,1
5553,9907907,https://www.airbnb.com/rooms/9907907,Large double room in Bondi,A bright large double room is available in a b...,Not Specified,A bright large 

In [5]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5555 entries, 0 to 5554
Data columns (total 66 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         5555 non-null   int64  
 1   listing_url                5555 non-null   object 
 2   name                       5555 non-null   object 
 3   summary                    5555 non-null   object 
 4   space                      5555 non-null   object 
 5   description                5555 non-null   object 
 6   neighborhood_overview      5555 non-null   object 
 7   notes                      5554 non-null   object 
 8   transit                    5555 non-null   object 
 9   access                     5555 non-null   object 
 10  interaction                5555 non-null   object 
 11  house_rules                5555 non-null   object 
 12  property_type              5555 non-null   object 
 13  room_type                  5555 non-null   objec

In [6]:
pip install pandas sqlalchemy pymysql



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
from sqlalchemy import create_engine

# Replace with your MySQL credentials and database name
USER = 'root'
PASSWORD = ''
HOST = 'localhost'
PORT = '3306'
DATABASE = 'airbnbinfo'

# Create SQLAlchemy engine for MySQL
engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

# Your DataFrame (df) that you want to insert
df = pd.read_csv('airbnbnew1.csv')  # Replace with your DataFrame source

# Insert DataFrame into MySQL table
# Replace 'your_table_name' with your actual table name
df.to_sql('airbnbdbt', con=engine, if_exists='append', index=False)


5555